In [19]:
from PA0 import PA0

In [2]:
import dask.dataframe as dd
import pandas as pd
from dask.distributed import Client
import json

In [3]:
{"number_products_rated": {"count": 996.0, "mean": 1.0, "std": 0.07, "min": 1.0, "25%": 1.0, "50%": 1.0, "75%": 1.0, "max": 3.0}, 
 "avg_ratings": {"count": 996.0, "mean": 4.16, "std": 1.25, "min": 1.0, "25%": 4.0, "50%": 5.0, "75%": 5.0, "max": 5.0}, 
 "reviewing_since": {"count": 996.0, "mean": 2011.63, "std": 3.01, "min": 1999.0, "25%": 2011.0, "50%": 2013.0, "75%": 2013.0, "max": 2014.0}, 
 "helpful_votes": {"count": 996.0, "mean": 1.96, "std": 8.81, "min": 0.0, "25%": 0.0, "50%": 0.0, "75%": 1.0, "max": 152.0}, 
 "total_votes": {"count": 996.0, "mean": 2.72, "std": 10.16, "min": 0.0, "25%": 0.0, "50%": 0.0, "75%": 2.0, "max": 157.0}}

{'number_products_rated': {'count': 996.0,
  'mean': 1.0,
  'std': 0.07,
  'min': 1.0,
  '25%': 1.0,
  '50%': 1.0,
  '75%': 1.0,
  'max': 3.0},
 'avg_ratings': {'count': 996.0,
  'mean': 4.16,
  'std': 1.25,
  'min': 1.0,
  '25%': 4.0,
  '50%': 5.0,
  '75%': 5.0,
  'max': 5.0},
 'reviewing_since': {'count': 996.0,
  'mean': 2011.63,
  'std': 3.01,
  'min': 1999.0,
  '25%': 2011.0,
  '50%': 2013.0,
  '75%': 2013.0,
  'max': 2014.0},
 'helpful_votes': {'count': 996.0,
  'mean': 1.96,
  'std': 8.81,
  'min': 0.0,
  '25%': 0.0,
  '50%': 0.0,
  '75%': 1.0,
  'max': 152.0},
 'total_votes': {'count': 996.0,
  'mean': 2.72,
  'std': 10.16,
  'min': 0.0,
  '25%': 0.0,
  '50%': 0.0,
  '75%': 2.0,
  'max': 157.0}}

In [4]:
# import timeit

# start = timeit.default_timer()

# PA0('user_reviews_Release.csv')

# stop = timeit.default_timer()

# print('Time: ', stop - start)  

In [25]:
df = dd.read_csv('small_dataset.csv').drop(['reviewText','summary','unixReviewTime','reviewerName'],axis=1)

In [6]:
df.compute()

,reviewerID,asin,helpful,overall,reviewTime
0,A2T0RJ91B0PQ03,B0016CRVLW,"[0, 0]",1.0,"05 21, 2014"
1,A3TYW0XA8HSGWB,B00EKR5S0Q,"[0, 0]",5.0,"04 27, 2014"
2,A2CME0TQU2IVVB,B001AUPJVO,"[1, 1]",5.0,"12 19, 2012"
3,A2E5IDLX7R388S,B000055Y57,"[0, 0]",5.0,"06 10, 2014"
4,A3CIEMYUGV6ZMR,0545265355,"[0, 0]",5.0,"04 11, 2012"
...,...,...,...,...,...
994,A2AZAO0FLD1NN6,0984032827,"[1, 1]",5.0,"10 29, 2012"
995,A303MONZTW2M4Y,B00009MGF7,"[15, 16]",4.0,"09 8, 2003"
996,A35G0ZAEJQA14R,B007TVUZB2,"[1, 2]",5.0,"10 4, 2012"
997,AWAI3GPWL5Y9H,B006WQE5S4,"[92, 106]",1.0,"07 25, 2012"


In [7]:
df['reviewing_since']=df['reviewTime'].str.slice(start=-4).astype('int16')
df['helpful_votes']=df['helpful'].str[1:-1].str.split(',').map(lambda x: x[0]).astype('int16')
df['total_votes']=df['helpful'].str[1:-1].str.split(',').map(lambda x: x[1]).astype('int16')

In [8]:
# ddf = df.repartition(10)

In [21]:
import timeit

start = timeit.default_timer()

% timeit -n 5 -r 2 PA0('small_dataset.csv')


stop = timeit.default_timer()

print('Time: ', stop - start)  

UsageError: Line magic function `%` not found.


In [24]:
%timeit -n 5 -r 2 PA0('small_dataset.csv')

/home/ubuntu/dask_env/lib/python3.6/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 45163 instead
  http_address["port"], self.http_server.port
/home/ubuntu/dask_env/lib/python3.6/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40691 instead
  http_address["port"], self.http_server.port
/home/ubuntu/dask_env/lib/python3.6/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33935 instead
  http_address["port"], self.http_server.port
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
/home/ubuntu/dask_env/lib/python3.6/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a c

4.43 s ± 1.78 s per loop (mean ± std. dev. of 2 runs, 5 loops each)


In [9]:

df.map_partitions(lambda x: x.groupby('reviewerID').agg({"asin": "count",'overall':'mean','reviewing_since':'min','helpful_votes':'sum','total_votes':'sum'})).describe().compute()


,asin,overall,reviewing_since,helpful_votes,total_votes
count,996.000000,996.000000,996.000000,996.000000,996.000000
mean,1.003012,4.160643,2011.625502,1.964859,2.722892
std,0.070824,1.252253,3.012088,8.812267,10.157529
min,1.000000,1.000000,1999.000000,0.000000,0.000000
25%,1.000000,4.000000,2011.000000,0.000000,0.000000
50%,1.000000,5.000000,2013.000000,0.000000,0.000000
75%,1.000000,5.000000,2013.000000,1.000000,2.000000
max,3.000000,5.000000,2014.000000,152.000000,157.000000


In [43]:
df['helpful'].str[1:-1].str.split(',').map(lambda x: x[1]).compute()

0         0
1         0
2         1
3         0
4         0
       ... 
994       1
995      16
996       2
997     106
998       2
Name: helpful, Length: 999, dtype: object

In [55]:
df.map_partitions(lambda df: df.assign(reviewing_since=df['reviewTime'].str.slice(start=-4),
                                       helpful_votes=df['helpful'].str[1:-1].str.split(',').map(lambda x: x[0]),
                                       total_votes=df['helpful'].str[1:-1].str.split(',').map(lambda x: x[-1]),
                                       number_products_rated=df['asin'],
                                       average_ratings=df['overall'], 
                                       meta={'reviewing_since':'int16','helpful_votes':'int16','total_votes':'int16',"number_products_rated":"int16","average_ratings":"int16"}))

ValueError: Metadata inference failed in `lambda`.

You have supplied a custom function and Dask is unable to 
determine the type of output that that function returns. 

To resolve this please provide a meta= keyword.
The docstring of the Dask function you ran should have more information.

Original error is below:
------------------------
ValueError('Length of values (5) does not match length of index (2)',)

Traceback:
---------
  File "/home/ubuntu/dask_env/lib/python3.6/site-packages/dask/dataframe/utils.py", line 180, in raise_on_meta_error
    yield
  File "/home/ubuntu/dask_env/lib/python3.6/site-packages/dask/dataframe/core.py", line 5332, in _emulate
    return func(*_extract_meta(args, True), **_extract_meta(kwargs, True))
  File "<ipython-input-55-c58ca852c2f3>", line 6, in <lambda>
    meta={'reviewing_since':'int16','helpful_votes':'int16','total_votes':'int16',"number_products_rated":"int16","average_ratings":"int16"}))
  File "/home/ubuntu/dask_env/lib/python3.6/site-pac

In [51]:
df['helpful'].str[1:-1].str.split(',').map(lambda x: x[1]).compute()

0         0
1         0
2         1
3         0
4         0
       ... 
994       1
995      16
996       2
997     106
998       2
Name: helpful, Length: 999, dtype: object

In [52]:
df['helpful'].str[1:-1].str.split(',').map(lambda x: x[-1]).compute()

0         0
1         0
2         1
3         0
4         0
       ... 
994       1
995      16
996       2
997     106
998       2
Name: helpful, Length: 999, dtype: object

In [54]:
df.map_partitions(lambda df: df.assign(reviewing_since=df['reviewTime'].str.slice(start=-4),
                                       total_votes=df['helpful'].str[1:-1].str.split(',').map(lambda x: x[-1]),
                                       meta={'reviewing_since':'int16','total_votes':'int16'}))

,reviewerID,asin,helpful,overall,reviewTime,reviewing_since,total_votes,meta
npartitions=1,,,,,,,,
,object,object,object,float64,object,object,object,object
,...,...,...,...,...,...,...,...


In [18]:
res.head()

ValueError: invalid literal for int() with base 10: 'A2T0RJ91B0PQ03'

In [ ]:
# ddf

In [ ]:
# ddf.compute()

In [ ]:
# df5 = df['helpful'].repartition(10)
# for i in range(df5.npartitions):
#     print(df5.partitions[i].compute())

In [ ]:
# import timeit

# start = timeit.default_timer()

# PA0('user_reviews_Release.csv')

# stop = timeit.default_timer()

# print('Time: ', stop - start)  